#**Google Drive Setup**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/TE_Mini_Project')

In [ ]:
%cd /content/gdrive/MyDrive/TE_Mini_Project

/content/gdrive/.shortcut-targets-by-id/1OEvlm8HHf_vOccqYESNlGmlmiZfQFptz/TE_Mini_Project


In [ ]:
%pwd

'/content/gdrive/.shortcut-targets-by-id/1OEvlm8HHf_vOccqYESNlGmlmiZfQFptz/TE_Mini_Project'

# **Nvidia GPU stats**

In [ ]:
!/usr/local/cuda/bin/nvcc --version

!nvidia-smi


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Tue Feb 15 16:06:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |            

# **1. Importing Libraries**

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-eu7utmwz
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-eu7utmwz
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=82af876daf987d52ce8ec195e88eaa999018023a963088ca051a108a270ee92e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9cd5cx81/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dense, Input, Conv2DTranspose, Dense, Flatten
from tensorflow.keras.layers import ReLU, Dropout, Concatenate, BatchNormalization, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, model_from_json
from keras import backend as K
import math
try:
    from keras_contrib.layers.normalization import InstanceNormalization
except Exception:
    from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
import numpy as np
from PIL import Image
import extraFunctions
import os
import sys

In [ ]:
tf.compat.v1.disable_eager_execution()
K.clear_session()
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


#**2. Loss functions**

In [ ]:
# Define the loss functions for the discriminator
def critic_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def gen_loss(fake_img,real_img):
    return -tf.reduce_mean(fake_img)


# **3. Critic Model**

In [ ]:
INPUT_SHAPE=(128,128,3)
MASK_PERCENTAGE=0.25
c_input_shape = (INPUT_SHAPE[0], int(INPUT_SHAPE[1] * (MASK_PERCENTAGE *2)), INPUT_SHAPE[2])
SHOW_SUMMARY=True

In [ ]:
def c_build_conv(layer_input, filters, kernel_size = 5, strides = 2, padding = 'same', activation='leakyrelu', dropout_rate=0.25, norm=True):
    conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(layer_input)
    if activation == 'leakyrelu':
        conv = LeakyReLU(alpha=0.2)(conv)
    if dropout_rate:
        conv = Dropout(rate=dropout_rate)(conv)
    if norm:
        conv = InstanceNormalization()(conv)
    return conv


#This returns the critic dense layer
def build_critic(d_input = Input(shape=c_input_shape)):
  with tf.compat.v1.variable_scope('D'):
    d = c_build_conv(d_input, 32, 5,strides=2, norm=False,activation='relu')

    d1 = c_build_conv(d, 64, 5, strides=2,activation='relu')
    d2 = c_build_conv(d1, 64, 5, strides=2,activation='relu')
    d3 = c_build_conv(d2, 64, 5, strides=2,activation='relu')
    d4 = c_build_conv(d3, 64, 5, strides=2,activation='relu')
    
    flat = Flatten()(d4)
    dense = Dense(512, activation='relu')(flat)
    
    
    return dense

In [ ]:
#This returns the complete critic model
def build_critic_1(d_input = Input(shape=c_input_shape)):
  with tf.compat.v1.variable_scope('D'):
    d = c_build_conv(d_input, 32, 5,strides=2, norm=False,activation='relu')

    d1 = c_build_conv(d, 64, 5, strides=2,activation='relu')
    d2 = c_build_conv(d1, 64, 5, strides=2,activation='relu')
    d3 = c_build_conv(d2, 64, 5, strides=2,activation='relu')
    d4 = c_build_conv(d3, 64, 5, strides=2,activation='relu')
    
    flat = Flatten()(d4)
    fc1 = Dense(512, activation='relu')(flat)
    #dense= Dense(1, activation='sigmoid')(fc1)
    
    
    return Model(d_input,fc1,name="critic")

In [ ]:
# Discriminator initialization
c_input=Input(shape=c_input_shape)
print(c_input.shape)
critic = build_critic_1(c_input)
critic_optimizer=Adam(0.0001,beta_1=0.5,beta_2=0.9)
#ct=Model(c_input,critic)
critic.compile(loss=critic_loss, optimizer=critic_optimizer)
if SHOW_SUMMARY:
    critic.summary()

(?, 128, 64, 3)
Model: "critic"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 64, 3)]      0         
                                                                 
 conv2d (Conv2D)             (None, 64, 32, 32)        2432      
                                                                 
 dropout (Dropout)           (None, 64, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 16, 64)        51264     
                                                                 
 dropout_1 (Dropout)         (None, 32, 16, 64)        0         
                                                                 
 instance_normalization (Ins  (None, 32, 16, 64)       2         
 tanceNormalization)                                             
                                            

#**4. Gradient Penalty to stabalize WGAN**

In [ ]:
def gradient_penalty(real_image, fake_image):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image (real + fake image)
        and added to the discriminator loss.
        """
        #tf.compat.v1.reset_default_graph()
        # Get the interpolated image
        alpha = tf.random.normal([16, 1, 1, 1], 0.0, 1.0)
        diff = fake_image - real_image
        interpolated = real_image + alpha * diff
        inter=Input(shape=[128,128,3],batch_size=16)


        
        pred_md = build_critic_1(d_input=inter)
        pred=pred_md(interpolated,training=True)
        # 2. Calculate the gradients w.r.t to this interpolated image.
        #tf.compat.v1.disable_v2_behavior()
        grads = tf.gradients(pred, [interpolated])
        # 3. Calculate the norm of the gradients.
        norm = tf.norm(grads[0],axis=1,ord='euclidean')
        gp = tf.reduce_mean(tf.square((norm - 1.0)))
        #tf.compat.v1.disable_v2_behavior()

       
        return gp


#**5. Generator Model**

In [ ]:
INPUT_SHAPE=(128,128,3)
MASK_PERCENTAGE=0.25
g_input_shape = (INPUT_SHAPE[0], int(INPUT_SHAPE[1] * (MASK_PERCENTAGE *2)), INPUT_SHAPE[2])
GEN_OPTIMIZER = Adam(0.0001, beta_1=0.5,beta_2=0.9)
SHOW_SUMMARY = True

In [ ]:
def g_build_conv(layer_input, filters, kernel_size, strides, padding='same', activation='relu', norm=True, dilation_rate=1):
    conv = Conv2D(filters, kernel_size=[kernel_size,kernel_size], strides=[strides,strides], dilation_rate=(dilation_rate, dilation_rate), padding=padding)(layer_input)
    if activation=='relu':
        conv = Activation('relu')(conv)
    if norm:
        conv = InstanceNormalization()(conv)
    return conv

def g_build_deconv(layer_input, filters, kernel_size, strides, padding='same', activation='relu'):
    deconv = Conv2DTranspose(filters, kernel_size = [kernel_size,kernel_size], strides = [strides,strides], padding = 'same')(layer_input)
    if activation == 'relu':
        deconv = Activation('relu')(deconv)
    return deconv

IMAGE_SZ=128
def build_generator(z):
    with tf.compat.v1.variable_scope('G'):
      ##### Encoder #####
      g1 = g_build_conv(z, 64, 5, strides=1)
      g2 = g_build_conv(g1, 128, 3, strides=2)
      g3 = g_build_conv(g2, 256, 3, strides=1)
      
      # Dilated Convolutions
      g4 = g_build_conv(g3, 256, 3, strides=1, dilation_rate=2)
      g5 = g_build_conv(g4, 256, 3, strides=1, dilation_rate=4)
      g6 = g_build_conv(g5, 256, 3, strides=1, dilation_rate=8)
      g7 = g_build_conv(g6, 256, 3, strides=1)

      #### Decoder ####
      g8 = g_build_deconv(g7, 128, 4, strides=2)
      g9 = g_build_conv(g8, 64, 3, strides=1)
      
      generator_output = Conv2D(3, kernel_size=3, strides=(1,1), activation='sigmoid', padding='same', dilation_rate=(1,1))(g9) 
    
    return generator_output

#**6. Concatenator**

In [ ]:
def concatenator(x_in):
  with tf.compat.v1.variable_scope('C'):
    dense_=Dense(units=1,activation='sigmoid')(x_in)
    return dense_
INPUT_SHAPE=(128,128,3)
MASK_PERCENTAGE=0.25
x_input_shape = (INPUT_SHAPE[0], int(INPUT_SHAPE[1] * (MASK_PERCENTAGE *2)), INPUT_SHAPE[2])
x_input=Input(shape=x_input_shape)
gen=build_critic(x_input)
cct= concatenator(x_input)
print(cct.shape)
concat= Model(x_input,cct)
concat.compile(loss=gen_loss,optimizer=GEN_OPTIMIZER)
if SHOW_SUMMARY:
  concat.summary()

(?, 128, 64, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 64, 3)]      0         
                                                                 
 dense_2 (Dense)             (None, 128, 64, 1)        4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


#**7. Train The Model**

In [ ]:
#tf.compat.v1.disable_v2_behavior()

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
def train_model ():
      #tf.compat.v1.disable_eager_execution()
      """ 
      function to train the WGAN model :
      we have consider three phase training for the same:
      1. Phase 1 train the generator
      2. Phase 2 train the discriminator(critic)
      3. Phase 3 train both of them.

      """

      start_iter = 201400
      #training hyperparameters
      BATCH_SZ = 16
      VERBOSE = True
      IMAGE_SZ = 128
      OUT_DIR = '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results'
      OUT_DIR_img='/content/gdrive/MyDrive/TE_Mini_Project/saved_imgs'
      OUT_DIR_test= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test'
      OUT_DIR_test_blend= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test1_blend'
      OUT_DIR_unblend= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test1_unblend'
      OUT_DIR_train='/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/train'
      MODEL_DIR = os.path.join(OUT_DIR, 'models')
      INFO_PATH = os.path.join(OUT_DIR, 'run.txt')
      N_TEST = 10
      N_ITERS = 10600 + start_iter
      N_ITERS_P1 = 38160 # How many iterations to train in phase 1 18% of total iterations
      N_ITERS_P2 = 4240 # How many iterations to train in phase 2 2% of total iterations
      INTV_PRINT = 1060 # How often to print
      INTV_SAVE = 1060 # How often to save the model
      lmd=10
      eps=0.0004

      #to check if we dont clobber the existing run
      if os.path.isdir(OUT_DIR) and len(os.listdir(OUT_DIR)) > 2:
        print('Warning, OUT_DIR already exists. Aborting.')
        exit()
      model_filename = None
      #load the model if iteration is mentioned
      if start_iter > 0:
        model_filename = "/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model201400.ckpt"
      G_Z = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 4], name='G_Z')
      D_X = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 3], name='D_X')

      """
      loading dataset part here

      """

      data = np.load('dataset_4300_split.npz')
      imgs = data['imgs_train'] 
      imgs_p = extraFunctions.preprocess_images_outpainting(imgs)

      test_imgs = data['imgs_test']
      test_imgs_p = extraFunctions.preprocess_images_outpainting(test_imgs)

      test_img = test_imgs[:N_TEST]
      test_img_p = test_imgs_p[:N_TEST]

      train_img = imgs[4, np.newaxis]
      train_img_p = imgs_p[4, np.newaxis]


      # Write training and testing sample ground truths as reference
      extraFunctions.save_image(train_img[0], os.path.join(OUT_DIR_train, 'train_img.png'))
      for i_test in range(N_TEST):
          extraFunctions.save_image(test_imgs[i_test], os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test))


      G_sample = build_generator(G_Z)
      vars_G = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='G')

      C_real = concatenator(build_critic(D_X))
      C_fake = concatenator(build_critic(G_sample))
      vars_D = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='D')
      vars_C = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='C')

      #calculating the generator loss for the input
      
      real_loss = tf.reduce_mean(C_real)
      fake_loss = tf.reduce_mean(C_fake)
      loss= fake_loss - real_loss
      C_loss = loss
      G_MSE_loss = tf.compat.v1.losses.mean_squared_error(G_sample, D_X, weights=tf.expand_dims(G_Z[:,:,:,3], -1))
      G_loss = G_MSE_loss-eps*tf.reduce_mean(C_fake)
      #caculating the gradient penalty for the input set
      print(D_X.shape)
      print(G_sample.shape)          
      gp = gradient_penalty(D_X,G_sample)
      
      #updaing the critic loss by adding the gradient penalty to it
      C_loss= C_loss+ gp*lmd

      #hyperparameters set according to the paper: Improved Training of WGANs
      G_MSE_opt= tf.compat.v1.train.AdamOptimizer().minimize(G_MSE_loss, var_list=vars_G)
      C_solver = tf.compat.v1.train.AdamOptimizer()
      G_solver = tf.compat.v1.train.AdamOptimizer()
      G_opt = G_solver.minimize(G_loss, var_list=vars_G)
      C_opt = C_solver.minimize(C_loss, var_list=(vars_D + vars_C))

      train_loss = []
      out_loss = []

      last_output_PATH = [None] * N_TEST 

      #assert N_ITERS > N_ITERS_P1 + N_ITERS_P2

      # Saver to save the session
      saver = tf.compat.v1.train.Saver()

      with tf.compat.v1.Session() as sess:
        if model_filename is None:
            sess.run(tf.compat.v1.global_variables_initializer())
        else:
            saver.restore(sess, model_filename)
        for i in range(start_iter, N_ITERS + 1):
            batch, batch_p = extraFunctions.sample_random_minibatch(imgs, imgs_p, BATCH_SZ)
            G_sample_ = None
            C_loss_curr, G_loss_curr, G_MSE_loss_curr = None, None, None
            if i < N_ITERS_P1: #Phase 1 Training the Generator network
              if i == 0:
                print('------------------> Beginning Phase 1...')
              _,G_MSE_loss_curr, G_sample_ = sess.run([G_MSE_opt,G_MSE_loss, G_sample], feed_dict={D_X: batch, G_Z: batch_p})
              

            elif i < N_ITERS_P1 + N_ITERS_P2:# Phase 2 Training the Critic network
              if i == N_ITERS_P1:
                print('------------------> Beginning Phase 2...')
              _, C_loss_curr, C_real_, C_fake_ = sess.run([C_opt, C_loss, C_real, C_fake], feed_dict={D_X: batch, G_Z: batch_p}) 
              if VERBOSE:
                print((i, C_loss_curr, np.min(C_real_), np.max(C_real_), np.min(C_fake_), np.max(C_fake_)))

            else: # Phase 3 - Training both Generator and Critic
              if i == N_ITERS_P1 + N_ITERS_P2:
                  print('------------------> Beginning Phase 3...')
              _, C_loss_curr, C_real_, C_fake_ = sess.run([C_opt, C_loss, C_real, C_fake], feed_dict={D_X: batch, G_Z: batch_p})
              if VERBOSE:
                  print((i, C_loss_curr, 'D', np.min(C_real_), np.max(C_real_), np.min(C_fake_), np.max(C_fake_)))
              _, G_loss_curr, G_MSE_loss_curr, G_sample_, C_fake_ = sess.run([G_opt, G_loss, G_MSE_loss, G_sample, C_fake], feed_dict={D_X: batch, G_Z: batch_p})
              if VERBOSE:
                  print((i, G_loss_curr, 'G', np.min(C_fake_), np.max(C_fake_)))
            
            # Periodically test the generator on held-out images
            if i % INTV_PRINT == 0:
                G_MSE_loss_curr_out = None
                if G_sample_ is not None:
                    # Print out the output image
                    output, G_MSE_loss_curr_out = sess.run([G_sample, G_MSE_loss], feed_dict={D_X: test_img, G_Z: test_img_p})
                    for i_test in range(N_TEST):
                        extraFunctions.save_image(output[i_test], os.path.join(OUT_DIR_img, 'out_%d_%d.png' % (i_test, i)))
                        last_output_PATH[i_test] = os.path.join(OUT_DIR_img, 'out_%d_%d.png' % (i_test, i))
                    # Also save the train image
                    output, = sess.run([G_sample], feed_dict={D_X: train_img, G_Z: train_img_p})
                    extraFunctions.save_image(output[0], os.path.join(OUT_DIR_img, 'train%d.png' % i))
                print('Iteration [%d/%d]:' % (i, N_ITERS))
                if G_MSE_loss_curr is not None:
                    print('\tG_MSE_loss (train) = %f' % G_MSE_loss_curr)
                if G_MSE_loss_curr_out is not None:
                    print('\tG_MSE_loss (out) = %f' % G_MSE_loss_curr_out)
                if G_loss_curr is not None:
                    print('\tG_loss = %f' % G_loss_curr)
                if C_loss_curr is not None:
                    print('\tC_loss = %f' % C_loss_curr)
              
            # Keep track of losses for logging
            if G_MSE_loss_curr is not None:
                train_loss.append([i, G_MSE_loss_curr])
            if G_MSE_loss_curr_out is not None:
                out_loss.append([i, G_MSE_loss_curr_out])

            # Save the model every so often
            if i % INTV_SAVE == 0:
                save_path = saver.save(sess, os.path.join(MODEL_DIR, 'model%d.ckpt' % i))
                print('Model saved in path: %s' % save_path)

            # Save the loss every so often
            if i % INTV_SAVE == 0:
                np.savez(os.path.join(OUT_DIR, f"loss{N_ITERS}.npz"), train_loss=np.array(train_loss), out_loss=np.array(out_loss))

            """
            code for saving the losses in a file and plotting them
            """
            # Save the loss
      np.savez(os.path.join(OUT_DIR, f'loss{N_ITERS}.npz'), train_loss=np.array(train_loss), out_loss=np.array(out_loss))
      #extraFunctions.plot_loss(os.path.join(OUT_DIR, 'loss.npz'), 'Loss During Training', os.path.join(OUT_DIR, 'loss_plot.png'))
      
      for i_test in range(N_TEST):
        extraFunctions.postprocess_images_outpainting(os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test), last_output_PATH[i_test], os.path.join(OUT_DIR_unblend, 'out_paste_%d.png' % i_test), blend=False)
        extraFunctions.postprocess_images_outpainting(os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test), last_output_PATH[i_test], os.path.join(OUT_DIR_test_blend, 'out_blend_%d.png' % i_test), blend=True)
      

In [ ]:
train_model()

Streaming output truncated to the last 5000 lines.
(209510, 6.672265, 'D', 1.0, 1.0, 1.4637595e-15, 4.9512325e-14)
(209510, 0.0026376385, 'G', 1.4637539e-15, 4.951204e-14)
(209511, 5.4666276, 'D', 1.0, 1.0, 2.143671e-15, 2.0723318e-14)
(209511, 0.0021021203, 'G', 2.143663e-15, 2.0723318e-14)
(209512, 6.2774053, 'D', 1.0, 1.0, 1.3744296e-15, 4.8842335e-14)
(209512, 0.0026981968, 'G', 1.3744245e-15, 4.884224e-14)
(209513, 5.564818, 'D', 1.0, 1.0, 1.6868669e-15, 1.3897867e-13)
(209513, 0.0025374526, 'G', 1.6868606e-15, 1.3897867e-13)
(209514, 6.1004777, 'D', 1.0, 1.0, 1.3448928e-15, 1.4659487e-13)
(209514, 0.002206431, 'G', 1.3448928e-15, 1.4659431e-13)
(209515, 7.0049257, 'D', 1.0, 1.0, 1.0786789e-15, 1.2497331e-13)
(209515, 0.0024467574, 'G', 1.0786789e-15, 1.2497258e-13)
(209516, 5.9242215, 'D', 1.0, 1.0, 1.7143139e-15, 5.3200093e-14)
(209516, 0.0022681986, 'G', 1.7143072e-15, 5.319989e-14)
(209517, 4.109869, 'D', 1.0, 1.0, 1.6373641e-15, 2.0021546e-14)
(209517, 0.0023369913, 'G', 1.63

#**8. Testing with unknown Image**

In [ ]:
import cv2
def postprocess_images_outpainting(img_PATH, img_o_PATH, out_PATH, blend=False): # img, img_0 are (64, 64, 3), mask is (64, 64, 1)
    src = cv2.imread(img_PATH)[:, int(2 * IMAGE_SZ / 8):-int(2 * IMAGE_SZ / 8), :]
    dst = cv2.imread(img_o_PATH)
    if blend:
        mask = np.ones(src.shape, src.dtype) * 255
        center = (int(IMAGE_SZ / 2) - 1, int(IMAGE_SZ / 2) - 1)
        out = cv2.seamlessClone(src, dst, mask, center, cv2.NORMAL_CLONE)
    else:
        out = dst.copy()
        out[:, int(2 * IMAGE_SZ / 8):-int(2 * IMAGE_SZ / 8), :] = src
    cv2.imwrite(out_PATH, out)

In [ ]:
def preprocess_images_outpainting(imgs, crop=True):
    m = imgs.shape[0]
    imgs = np.array(imgs, copy=True)
    pix_avg = np.mean(imgs, axis=(1, 2, 3))
    if crop:
        imgs[:, :, :int(2 * IMAGE_SZ / 8), :] = imgs[:, :, int(-2 * IMAGE_SZ / 8):, :] = pix_avg[:, np.newaxis, np.newaxis, np.newaxis]
    mask = np.zeros((m, IMAGE_SZ, IMAGE_SZ, 1))
    mask[:, :, :int(2 * IMAGE_SZ / 8), :] = mask[:, :, int(-2 * IMAGE_SZ / 8):, :] = 1.0
    imgs_p = np.concatenate((imgs, mask), axis=3)
    return imgs_p

In [ ]:
def norm_image(img_r):
    img_norm = (img_r * 255.0).astype(np.uint8)
    return img_norm
def save_image(img_r, name, mode='RGB'):
    img_norm = norm_image(img_r)
    img = Image.fromarray(img_norm, mode)
    img= img.convert("RGB")
    img.save(name, format='PNG')

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import extraFunctions 
from keras import backend as K

#Before prediction
K.clear_session()
tf.compat.v1.reset_default_graph()
in_PATH="/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test/test_img_6.png"
out_PATH="/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/out.png"
new_PATH="/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/postprocess.png"

img = np.array(Image.open(in_PATH).convert('RGB'))[np.newaxis] / 255.0
img_p = preprocess_images_outpainting(img)
global loaded_graph
loaded_graph = tf.compat.v1.Graph()
with loaded_graph.as_default():
  G_Z = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 4], name='G_Z')
  D_X = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 3], name='D_X')
  G_in = build_generator(G_Z)
    

  checkpoint_path="/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model212000.ckpt"
  #variables = tf.train.list_variables(checkpoint_path)
  with tf.compat.v1.Session(graph=loaded_graph) as sess:
      #saver = tf.compat.v1.train.import_meta_graph('/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model212000.ckpt.meta')
      saveables = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='G')
      saver = tf.compat.v1.train.Saver(saveables)
      saver.restore(sess, checkpoint_path)
      output, = sess.run([G_in], feed_dict={G_Z: img_p})
      save_image(output[0], out_PATH)
      postprocess_images_outpainting(in_PATH,out_PATH,new_PATH)
      K.clear_session()

In [ ]:
def test_model ():
      #tf.compat.v1.disable_eager_execution()
      """ 
      function to train the WGAN model :
      we have consider three phase training for the same:
      1. Phase 1 train the discriminator(critic)
      2. Phase 2 train the generator 
      3. Phase 3 train both of them.

      """

      start_iter = 212000
      #training hyperparameters
      BATCH_SZ = 16
      VERBOSE = True
      IMAGE_SZ = 128
      OUT_DIR = '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results'
      OUT_DIR_img='/content/gdrive/MyDrive/TE_Mini_Project/saved_imgs'
      OUT_DIR_test= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test'
      OUT_DIR_test_blend= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test1_blend'
      OUT_DIR_unblend= '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/test1_unblend'
      OUT_DIR_train='/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/train'
      MODEL_DIR = os.path.join(OUT_DIR, 'models')
      #INFO_PATH = os.path.join(OUT_DIR, 'run.txt')
      N_TEST = 20
      N_ITERS = 5 + start_iter
      N_ITERS_P1 = 38160 # How many iterations to train in phase 1 18% of total iterations
      N_ITERS_P2 = 4240 # How many iterations to train in phase 2 2% of total iterations
      INTV_PRINT = 1060 # How often to print
      INTV_SAVE = 5 # How often to save the model
      lmd=10
      eps=0.0004

      #to check if we dont clobber the existing run
      if os.path.isdir(OUT_DIR) and len(os.listdir(OUT_DIR)) > 2:
        print('Warning, OUT_DIR already exists. Aborting.')
        exit()
      model_filename = None
      #load the model if iteration is mentioned
      if start_iter > 0:
        model_filename = '/content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model212000.ckpt'
      G_Z = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 4], name='G_Z')
      D_X = tf.compat.v1.placeholder(tf.float32, shape=[None, IMAGE_SZ, IMAGE_SZ, 3], name='D_X')

      """
      loading dataset part here

      """

      data = np.load('dataset_4300_split.npz')
      imgs = data['imgs_train'] 
      imgs_p = extraFunctions.preprocess_images_outpainting(imgs)

      test_imgs = data['imgs_test']
      test_imgs_p = extraFunctions.preprocess_images_outpainting(test_imgs)

      test_img = test_imgs[:N_TEST]
      test_img_p = test_imgs_p[:N_TEST]

      train_img = imgs[4, np.newaxis]
      train_img_p = imgs_p[4, np.newaxis]


      # Write training and testing sample ground truths as reference
      extraFunctions.save_image(train_img[0], os.path.join(OUT_DIR_train, 'train_img.png'))
      for i_test in range(N_TEST):
          extraFunctions.save_image(test_imgs[i_test], os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test))


      G_sample = build_generator(G_Z)
      vars_G = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='G')

      C_real = concatenator(build_critic(D_X))
      C_fake = concatenator(build_critic(G_sample))
      vars_D = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='D')
      vars_C = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope='C')

      #calculating the generator loss for the input
      
      real_loss = tf.reduce_mean(C_real)
      fake_loss = tf.reduce_mean(C_fake)
      loss= fake_loss - real_loss
      C_loss = loss
      G_MSE_loss = tf.compat.v1.losses.mean_squared_error(G_sample, D_X, weights=tf.expand_dims(G_Z[:,:,:,3], -1))
      G_loss = G_MSE_loss-eps*tf.reduce_mean(C_fake)
      #caculating the gradient penalty for the input set
      print(D_X.shape)
      print(G_sample.shape)          
      gp = gradient_penalty(D_X,G_sample)
      
      #updaing the critic loss by adding the gradient penalty to it
      C_loss= C_loss+ gp*lmd

      #hyperparameters set according to the paper: Improved Training of WGANs
      G_MSE_opt= tf.compat.v1.train.AdamOptimizer().minimize(G_MSE_loss, var_list=vars_G)
      C_solver = tf.compat.v1.train.AdamOptimizer()
      G_solver = tf.compat.v1.train.AdamOptimizer()
      G_opt = G_solver.minimize(G_loss, var_list=vars_G)
      C_opt = C_solver.minimize(C_loss, var_list=(vars_D + vars_C))

      train_loss = []
      out_loss = []

      last_output_PATH = [None] * N_TEST 

      #assert N_ITERS > N_ITERS_P1 + N_ITERS_P2

      # Saver to save the session
      saver = tf.compat.v1.train.Saver()

      with tf.compat.v1.Session() as sess:
        if model_filename is None:
            sess.run(tf.compat.v1.global_variables_initializer())
        else:
            saver.restore(sess, model_filename)
        for i in range(start_iter, N_ITERS + 1):
            batch, batch_p = extraFunctions.sample_random_minibatch(imgs, imgs_p, BATCH_SZ)
            G_sample_ = None
            C_loss_curr, G_loss_curr, G_MSE_loss_curr = None, None, None
            if i < N_ITERS_P1: #Phase 1 Training the Generator network
              if i == 0:
                print('------------------> Beginning Phase 1...')
              _,G_MSE_loss_curr, G_sample_ = sess.run([G_MSE_opt,G_MSE_loss, G_sample], feed_dict={D_X: batch, G_Z: batch_p})
              

            elif i < N_ITERS_P1 + N_ITERS_P2:# Phase 2 Training the Critic network
              if i == N_ITERS_P1:
                print('------------------> Beginning Phase 2...')
              _, C_loss_curr, C_real_, C_fake_ = sess.run([C_opt, C_loss, C_real, C_fake], feed_dict={D_X: batch, G_Z: batch_p}) 
              if VERBOSE:
                print((i, C_loss_curr, np.min(C_real_), np.max(C_real_), np.min(C_fake_), np.max(C_fake_)))

            else: # Phase 3 - Training both Generator and Critic
              if i == N_ITERS_P1 + N_ITERS_P2:
                  print('------------------> Beginning Phase 3...')
              _, C_loss_curr, C_real_, C_fake_ = sess.run([C_opt, C_loss, C_real, C_fake], feed_dict={D_X: batch, G_Z: batch_p})
              if VERBOSE:
                  print((i, C_loss_curr, 'D', np.min(C_real_), np.max(C_real_), np.min(C_fake_), np.max(C_fake_)))
              _, G_loss_curr, G_MSE_loss_curr, G_sample_, C_fake_ = sess.run([G_opt, G_loss, G_MSE_loss, G_sample, C_fake], feed_dict={D_X: batch, G_Z: batch_p})
              if VERBOSE:
                  print((i, G_loss_curr, 'G', np.min(C_fake_), np.max(C_fake_)))
            
            # Periodically test the generator on held-out images
            if i % INTV_PRINT == 0:
                G_MSE_loss_curr_out = None
                if G_sample_ is not None:
                    # Print out the output image
                    output, G_MSE_loss_curr_out = sess.run([G_sample, G_MSE_loss], feed_dict={D_X: test_img, G_Z: test_img_p})
                    for i_test in range(N_TEST):
                        extraFunctions.save_image(output[i_test], os.path.join(OUT_DIR_img, 'out_%d_%d.png' % (i_test, i)))
                        last_output_PATH[i_test] = os.path.join(OUT_DIR_img, 'out_%d_%d.png' % (i_test, i))
                    # Also save the train image
                    output, = sess.run([G_sample], feed_dict={D_X: train_img, G_Z: train_img_p})
                    extraFunctions.save_image(output[0], os.path.join(OUT_DIR_img, 'train%d.png' % i))
                print('Iteration [%d/%d]:' % (i, N_ITERS))
                if G_MSE_loss_curr is not None:
                    print('\tG_MSE_loss (train) = %f' % G_MSE_loss_curr)
                if G_MSE_loss_curr_out is not None:
                    print('\tG_MSE_loss (out) = %f' % G_MSE_loss_curr_out)
                if G_loss_curr is not None:
                    print('\tG_loss = %f' % G_loss_curr)
                if C_loss_curr is not None:
                    print('\tC_loss = %f' % C_loss_curr)
              
            # Keep track of losses for logging
            if G_MSE_loss_curr is not None:
                train_loss.append([i, G_MSE_loss_curr])
            if G_MSE_loss_curr_out is not None:
                out_loss.append([i, G_MSE_loss_curr_out])

            # Save the model every so often
            if i % INTV_SAVE == 0:
                save_path = saver.save(sess, os.path.join(MODEL_DIR, 'model%d.ckpt' % i))
                print('Model saved in path: %s' % save_path)

            # Save the loss every so often
            if i % INTV_SAVE == 0:
                np.savez(os.path.join(OUT_DIR, f"loss{N_ITERS}.npz"), train_loss=np.array(train_loss), out_loss=np.array(out_loss))

            """
            code for saving the losses in a file and plotting them
            """
            # Save the loss
      np.savez(os.path.join(OUT_DIR, f'loss{N_ITERS}.npz'), train_loss=np.array(train_loss), out_loss=np.array(out_loss))
      #extraFunctions.plot_loss(os.path.join(OUT_DIR, 'loss.npz'), 'Loss During Training', os.path.join(OUT_DIR, 'loss_plot.png'))
      
      for i_test in range(N_TEST):
        extraFunctions.postprocess_images_outpainting(os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test), last_output_PATH[i_test], os.path.join(OUT_DIR_unblend, 'out_paste_%d.png' % i_test), blend=False)
        extraFunctions.postprocess_images_outpainting(os.path.join(OUT_DIR_test, 'test_img_%d.png' % i_test), last_output_PATH[i_test], os.path.join(OUT_DIR_test_blend, 'out_blend_%d.png' % i_test), blend=True)
      

In [ ]:
test_model()

Warning, OUT_DIR already exists. Aborting.
(?, 128, 128, 3)
(?, 128, 128, 3)
INFO:tensorflow:Restoring parameters from /content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model212000.ckpt
(212000, 7.301627, 'D', 1.0, 1.0, 1.5153255e-15, 3.3536623e-14)
(212000, 0.002498639, 'G', 1.5153198e-15, 3.3536623e-14)
Iteration [212000/212005]:
	G_MSE_loss (train) = 0.002499
	G_MSE_loss (out) = 0.022923
	G_loss = 0.002499
	C_loss = 7.301627
Model saved in path: /content/gdrive/MyDrive/TE_Mini_Project/WGAN_model/results/models/model212000.ckpt
(212001, 8.495618, 'D', 1.0, 1.0, 1.5661074e-15, 1.776001e-14)
(212001, 0.002374483, 'G', 1.5661015e-15, 1.776001e-14)
(212002, 4.734019, 'D', 1.0, 1.0, 1.2207049e-15, 2.0459786e-14)
(212002, 0.002402275, 'G', 1.2207004e-15, 2.0459745e-14)
(212003, 8.169338, 'D', 1.0, 1.0, 1.802166e-15, 5.8267274e-14)
(212003, 0.0026387675, 'G', 1.8021592e-15, 5.8266936e-14)
(212004, 13.046668, 'D', 1.0, 1.0, 2.2175138e-15, 3.493808e-14)
(212004, 0.00291734, 'G

#**Calculate RMSE**

In [ ]:
def compute_RMSE(image_gt_PATH, image_o_PATH):
    im_gt = np.array(Image.open(image_gt_PATH).convert('RGB')).astype(np.float64)
    im_o = np.array(Image.open(image_o_PATH).convert('RGB')).astype(np.float64)
    assert im_gt.shape == (128, 128, 3)
    assert im_o.shape == (128, 128, 3)
    M = np.ones((128, 128, 3))
    M[:, 32:96, :] = 0
    num_pixels = 128 * 64 * 3
    return np.sqrt(np.sum(((im_gt - im_o) * M) ** 2) / num_pixels)

In [ ]:
in_PATH,out_PATH="/content/gdrive/MyDrive/TE_Mini_Project/Nature_0.jpg",'/content/gdrive/MyDrive/TE_Mini_Project/test.png'
res=compute_RMSE(in_PATH,out_PATH)
print(res)